In [1]:
from helium import *
from bs4 import BeautifulSoup

start_chrome('https://millercenter.org/the-presidency/presidential-speeches', headless=False)
wait_until(lambda: S('.option') is not None)


page_source = get_driver().page_source


soup = BeautifulSoup(page_source, 'html.parser')

# Find all speech titles under the "option" class
speech_titles = [label.get_text() for label in soup.find_all('label', class_='option')]

print(speech_titles)


['George Washington', 'John Adams', 'Thomas Jefferson', 'James Madison', 'James Monroe', 'John Quincy Adams', 'Andrew Jackson', 'Martin Van Buren', 'William Harrison', 'John Tyler', 'James K. Polk', 'Zachary Taylor', 'Millard Fillmore', 'Franklin Pierce', 'James Buchanan', 'Abraham Lincoln', 'Andrew Johnson', 'Ulysses S. Grant', 'Rutherford B. Hayes', 'James A. Garfield', 'Chester A. Arthur', 'Grover Cleveland', 'Benjamin Harrison', 'William McKinley', 'Theodore Roosevelt', 'William Taft', 'Woodrow Wilson', 'Warren G. Harding', 'Calvin Coolidge', 'Herbert Hoover', 'Franklin D. Roosevelt', 'Harry S. Truman', 'Dwight D. Eisenhower', 'John F. Kennedy', 'Lyndon B. Johnson', 'Richard M. Nixon', 'Gerald Ford', 'Jimmy Carter', 'Ronald Reagan', 'George H. W. Bush', 'Bill Clinton', 'George W. Bush', 'Barack Obama', 'Donald Trump', 'Joe Biden']


In [2]:
speech_titles[0]

'George Washington'

In [3]:
import time
import os
from helium import start_chrome, go_to, S, find_all, write, press, ENTER, click, wait_until, kill_browser

speech_dict={}

for i in range(len(speech_titles)):
    president = speech_titles[i]
    # Create a folder for the president if it doesn't exist
    if not os.path.exists(president):
        os.makedirs(president)
    
    # Go to the main page and search for the president's speech
    go_to('https://millercenter.org/the-presidency/presidential-speeches')
    write(president, into=S('#edit-field-full-node-value'))
    print(f"Searching for: {president}")
    press(ENTER)
    time.sleep(5)
    # Find all speech links on the results page
    speech_links = find_all(S('div.views-field-title a'))
    links = [speech.web_element.get_attribute("href") for speech in speech_links]
    speech_dict[president] = links

kill_browser()


Searching for: George Washington


KeyboardInterrupt: 

In [94]:
#save dictionary to a file
import json
with open('speech_dict.json', 'w') as file:
    json.dump(speech_dict, file)

In [5]:
#read dictionary from a file
import json
with open('speech_dict.json', 'r') as file:
    speech_dict = json.load(file)

In [6]:
import os
import time
from helium import start_chrome, click, S, wait_until, kill_browser, get_driver

for president in list(speech_dict.keys()):
    links = speech_dict[president]
    print(len(links))   

    for link in links:
        # Extract the title of the speech
        title = link.split('/')[-1]
        print(f"Processing speech: {title}")
        
        # Define the file path for the transcript
        file_path = os.path.join(president, f'{title}.txt')
        
        # Check if the file already exists
        if os.path.exists(file_path):
            print(f"File {file_path} already exists, skipping extraction.")
            continue  # Skip to the next speech if the file exists
        
        # Visit the speech page
        start_chrome(link, headless=True)
        
        # Handle pop-up if it exists
        time.sleep(5)  # Allow time for the pop-up to appear
        try:
            # Check if the pop-up close button is visible
            if S(".block-email-list-promo-modal-close").exists():
                print("Pop-up detected, attempting to close...")
                # Click the 'close' button to dismiss the pop-up
                click(S(".block-email-list-promo-modal-close"))
                print("Pop-up closed.")
        except Exception as e:
            print(f"Error closing pop-up: {e}")
        
        # Click to expand the transcript
        try:
            # Wait for the "View Transcript" button to appear
            wait_until(lambda: S('a.expandable-text-trigger').exists(), timeout_secs=15)

            # Use Selenium's execute_script to scroll down by 200 pixels
            driver = get_driver()  # Get the underlying Selenium driver
            driver.execute_script("window.scrollBy(0, 200);")
            time.sleep(1)  # Small delay to let the page scroll

            # Click the "View Transcript" button
            click(S('a.expandable-text-trigger'))
            print("Clicked on 'View Transcript'")
        except Exception as e:
            print(f"Error clicking on 'View Transcript': {e}")
        
        time.sleep(5)
        
        # Wait for the transcript to appear
        try:
            wait_until(lambda: S(".transcript-inner").exists(), timeout_secs=15)
            # Extract the transcript text
            transcript_element = S(".transcript-inner")
            transcript_text = transcript_element.web_element.text
        except Exception as e:
            print(f"Error extracting transcript: {e}")
            kill_browser()
            continue  # Move to the next link in case of failure
        
        # Save the transcript to a file inside the president's folder
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(transcript_text)
        print(f"Transcript saved as {file_path}")
        
        # Close the browser
        kill_browser()


32
Processing speech: january-5-2024-speech-third-anniversary-january-6th-attack
File George Washington\january-5-2024-speech-third-anniversary-january-6th-attack.txt already exists, skipping extraction.
Processing speech: january-20-2021-inaugural-address
File George Washington\january-20-2021-inaugural-address.txt already exists, skipping extraction.
Processing speech: july-4-2020-remarks-salute-america
File George Washington\july-4-2020-remarks-salute-america.txt already exists, skipping extraction.
Processing speech: june-20-2020-campaign-rally-tulsa-oklahoma
File George Washington\june-20-2020-campaign-rally-tulsa-oklahoma.txt already exists, skipping extraction.
Processing speech: june-13-2020-address-west-point-graduation
File George Washington\june-13-2020-address-west-point-graduation.txt already exists, skipping extraction.
Processing speech: june-1-2020-statement-protests-against-police-brutality
File George Washington\june-1-2020-statement-protests-against-police-brutality.

KeyboardInterrupt: 